In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, AutoPeftModelForCausalLM

/storage/ice1/2/3/ckniffin6/conda_envs/hf/lib/python3.12/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
from huggingface_hub import login
login()

In [3]:
model = AutoPeftModelForCausalLM.from_pretrained("/home/hice1/ckniffin6/scratch/DL/project/adapters/Qlora_rank16").to("cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# Load the saved model
modelName = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(modelName)

base_prompt_template: str = """
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: {query}
    
    Output:
"""
    

In [5]:
queries = ["If I take out a $35000 loan with a 5% APR how many $500 payments would it take to pay it back?",
           "Assuming an inital investment of $100,000, an expected cash flow of $7000 per month, an interest rate of 7% and 3 years before the project is complete what is the net present value?",
           "You want to save up over 4 years for you child's college. If the cost of attendance is $100000 and you deposit money each month into a savings account with a 4% annual interest rate, how much do you have to deposit each month?",
           "How does a business compute net income?",
           "What is the rule of 72?",
           "What is CPI and how does it tell us what the inflation rate is?"]    

In [6]:
summary1 = '"As of March 31, 2024, the Company had aggregate outstanding notes of $14,016 million, net of $61 million of issuance costs, with varying maturities (the "Notes"). Of the outstanding balance, $799 million, net of issuance costs, is classified as short-term debt on the Consolidated Balance Sheets. As of December 31, 2023, the Company had aggregate outstanding notes of $14,543 million, net of $65 million of issuance costs. Each of the Notes were issued at par and are senior unsecured obligations of the Company. Interest is payable semi-annually at fixed rates. A portion of the outstanding Notes is denominated in foreign currency (comprised of €5,170 million) and is remeasured into U.S. dollars at each balance sheet date (with remeasurement gain totaling $131 million for the three months ended March 31, 2024)."  Based on the preceding text how much short term debt does the company have?'

In [7]:
summary2 = '"We anticipate our full-year 2024 capital expenditures will be in the range of $30-37 billion, a $2 billion increase of the high end of our prior range. We expect growth will be driven by investments in servers, including both AI and non-AI hardware, and data centers as we ramp up construction on sites with our previously announced new data center architecture. Our updated outlook reflects our evolving understanding of our artificial intelligence (AI) capacity demands as we anticipate what we may need for the next generations of foundational research and product development. While we are not providing guidance for years beyond 2024, we expect our ambitious long-term AI research and product development efforts will require growing infrastructure investments beyond this year." Based on this text what are some risks the that could cause its expenditures to be greater than the predicted $2 billion?'

In [8]:
summary3 = '"Product Liability We recorded liabilities of $596 million and $561 million in Accrued liabilities and Other liabilities at September 30, 2023 and December 31, 2022 for the expected cost of all known product liability claims, plus an estimate of the expected cost for product liability claims that have already been incurred and are expected to be filed in the future for which we are self-insured. It is reasonably possible that our accruals for product liability claims may increase in future periods in material amounts, although we cannot estimate a reasonable range of incremental loss based on currently available information. We believe that any judgment against us involving our products for actual damages will be adequately covered by our recorded accruals and, where applicable, excess liability insurance coverage." How much did the company accrue in total liabilities according to the preceeding text?'

In [9]:
query = base_prompt_template.format(query=queries[0])

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: If I take out a $35000 loan with a 5% APR how many $500 payments would it take to pay it back?
    
    Output:
<s> 75 payments, or three years.  Your simple interest repayment will be ~$5,425: Principal * (1 + .05/12)^(12*years)-1) = 35000*(1.0025^-75) - 1, or ~47,424-35,000 = 12,424.  Divide this by 7.5 to get $1,659 per month.  As a practical matter, the repayments are likely to be round numbers, so you'll likely make 36 or 48 monthly payments.  That will add a bit to the ultimate cost due to accrued interest.  This assumes compounding monthly, but really you're probably looking at at the end of 48 months and not at each month so compounding annually is actually more likely.  Also, I have a vague sense that lenders like to tack on more than the interest rate to the loan.  I don't know if this is a u

In [10]:
query = base_prompt_template.format(query=queries[1])

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: Assuming an inital investment of $100,000, an expected cash flow of $7000 per month, an interest rate of 7% and 3 years before the project is complete what is the net present value?
    
    Output:
<s> The net present value of the cash flow is $159,639.71. This means that the present value of the cash flows is greater than the initial investment of $100,000, making the project a profitable investment. The net present value can be calculated using the following formula: NPV = -100,000 + (7000 * (1/1.07)^1) + (7000 * (1.07)^2) + (7000 * (1.07)^3) = $159,639.71  Where n is the number of years, r is the interest rate per year and PV is the present value. To learn more about how to calculate the net present value, refer to the following article: https://www.investopedia.com/articles/financial-analysis/1212

In [11]:
query = base_prompt_template.format(query=queries[2])

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: You want to save up over 4 years for you child's college. If the cost of attendance is $100000 and you deposit money each month into a savings account with a 4% annual interest rate, how much do you have to deposit each month?
    
    Output:
<s> $1565.55 per month. It takes 30 months to save $100,000 with a 1.5% per month deposit earning a 4% annual return. Monthly deposits is calculated from $100,000 / (1.015^30) = $1565.55 per month. Yearly deposits is calculated from $100,000 / (1.015) ^ 30 = $5262.22 per year.  Monthly deposit calculation is from https://www.calculator.net/interest-a-compounding-calculator.html Monthly deposit calculation is from http://www.financialcalculator.net/savings-calculator.html#savings-interest-calculator Monthly deposit calculation is from http://www.calculatorshop.com

In [12]:
query = base_prompt_template.format(query=queries[3])

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: How does a business compute net income?
    
    Output:
<s> Net income is the amount of money that a business has available after subtracting expenses from its income. To calculate a business's net income, begin by deducting the total expenses incurred over a specific period of time from the total amount of money earned during that same period. This results in the business's gross income. Next, subtract any deductions, such as depreciation and taxes, from the gross income, resulting in the business's net income. The net income can then be compared against industry standards for the same type of business to determine the performance of the company.  It is important to note that the net income of a business can also be influenced by non-cash items, such as accounting changes and one-time events. Therefo

In [13]:
query = base_prompt_template.format(query=queries[4])

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: What is the rule of 72?
    
    Output:
<s> It is used to calculate how long it will take for an investment to double in value. The rule of 72 says that your investment will double in value in 72/interest rate (in decimal form) years.  So if you have an investment earning 8% interest, it'll double in ~9 years (72/0.08=9.00). Edit: In terms of compounding (either periodic or continuous) -  http://www.calculator.net/financial-calculators/compound-interest-calculator.html  The formula (from the website) is:  FV = PV * ( 1 + (r/c) ) ^ t Or:  PV = (FV / (1 + (r/c) ^ t ))  This can be used to determine the initial deposit required to double one's investment. If we assume a 20% annual interest rate, and a continuous compounding scheme, then the doubling time will be 3.5688 years (72/20)  I'm assuming this is

In [14]:
query = base_prompt_template.format(query=queries[5])

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: What is CPI and how does it tell us what the inflation rate is?
    
    Output:
<s> The Consumer Price Index measures the average change over time in the prices paid by urban consumers for a representative collection of consumer goods and services. It also does the same calculation with respect to the prices paid by urban wage earners and clerical workers. Source: Wikipedia The problem is the "representative collection of consumer goods and services" is not representative at all. We have two sets of consumer goods here. One used by wage earners, the other by ... well the definition doesn't know the answer. So the index includes what a person earning a relatively low median salary (like a secretary purchasing a dress to wear to work) is going to spend the money on. It is meant to be a relative measure 

In [15]:
query = base_prompt_template.format(query=summary1)

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: "As of March 31, 2024, the Company had aggregate outstanding notes of $14,016 million, net of $61 million of issuance costs, with varying maturities (the "Notes"). Of the outstanding balance, $799 million, net of issuance costs, is classified as short-term debt on the Consolidated Balance Sheets. As of December 31, 2023, the Company had aggregate outstanding notes of $14,543 million, net of $65 million of issuance costs. Each of the Notes were issued at par and are senior unsecured obligations of the Company. Interest is payable semi-annually at fixed rates. A portion of the outstanding Notes is denominated in foreign currency (comprised of €5,170 million) and is remeasured into U.S. dollars at each balance sheet date (with remeasurement gain totaling $131 million for the three months ended March 31, 2

In [16]:
query = base_prompt_template.format(query=summary2)

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: "We anticipate our full-year 2024 capital expenditures will be in the range of $30-37 billion, a $2 billion increase of the high end of our prior range. We expect growth will be driven by investments in servers, including both AI and non-AI hardware, and data centers as we ramp up construction on sites with our previously announced new data center architecture. Our updated outlook reflects our evolving understanding of our artificial intelligence (AI) capacity demands as we anticipate what we may need for the next generations of foundational research and product development. While we are not providing guidance for years beyond 2024, we expect our ambitious long-term AI research and product development efforts will require growing infrastructure investments beyond this year." Based on this text what are

In [17]:
query = base_prompt_template.format(query=summary3)

modelInput = tokenizer(query, return_tensors="pt").to("cuda")

outputs = model.generate(**modelInput, max_new_tokens = 500, do_sample = True)
print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    You are a financial analyst designed to answer questions about business and finance. Your job is to reply to questions about finance topics and provide advice.

    Instruction: "Product Liability We recorded liabilities of $596 million and $561 million in Accrued liabilities and Other liabilities at September 30, 2023 and December 31, 2022 for the expected cost of all known product liability claims, plus an estimate of the expected cost for product liability claims that have already been incurred and are expected to be filed in the future for which we are self-insured. It is reasonably possible that our accruals for product liability claims may increase in future periods in material amounts, although we cannot estimate a reasonable range of incremental loss based on currently available information. We believe that any judgment against us involving our products for actual damages will be adequately covered by our recorded accruals and, where applicable, excess liability insura